# Text Classification with TensorFlow, Keras, and Cleanlab


In this quick-start tutorial, we use `cleanlab` to find potential label errors in the IMDb movie review text classification dataset. This dataset contains 50,000 text reviews, each labeled with a binary sentiment polarity label indicating whether the review is positive (1) or negative (0). `cleanlab` will shortlist _hundreds_ of examples that confuse our ML model the most; many of which are potential label errors, edge cases, or otherwise ambiguous examples.

**Overview of what we'll do in this tutorial:**

- Build a simple TensorFlow & Keras neural net and wrap it with SciKeras to make it scikit-learn compatible.

- Use this classifier to compute out-of-sample predicted probabilities, `pred_probs`, via cross validation.

- Identify potential label errors in the data with `cleanlab`'s `find_label_issues` method.

- Train a more robust version of the same neural net via `cleanlab`'s `LearningWithNoisyLabels` wrapper.

**Data:** https://ai.stanford.edu/~amaas/data/sentiment/


## **1. Install required dependencies**


Install the following dependencies with `pip install`:

1. cleanlab
2. sklearn
3. pandas
4. tensorflow
5. tensorflow-datasets
6. scikeras


In [ ]:
dependencies = ["cleanlab", "sklearn", "pandas", "tensorflow", "tensorflow_datasets", "scikeras"]

if "google.colab" in str(get_ipython()):  # Check if it's running in Google Colab
    %pip install cleanlab  # for colab
    cmd = ' '.join([dep for dep in dependencies if dep != "cleanlab"])
    %pip install $cmd
else:
    missing_dependencies = []
    for dependency in dependencies:
        try:
            __import__(dependency)
        except ImportError:
            missing_dependencies.append(dependency)

    if len(missing_dependencies) > 0:
        print("Missing required dependencies:")
        print(*missing_dependencies, sep=", ")
        print("\nPlease install them before running the rest of this notebook.")

In [ ]:
import os
import random
import numpy as np

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # Controls amount of tensorflow output

SEED = 123456  # Just for reproducibility

np.random.seed(SEED)
random.seed(SEED)

## **2. Load and preprocess the IMDb text dataset**


This dataset is provided in TensorFlow's Datasets.


In [ ]:
%%capture

import tensorflow_datasets as tfds

raw_full_ds = tfds.load(
    name="imdb_reviews", split=("train+test"), batch_size=-1, as_supervised=True
)
raw_full_texts, full_labels = tfds.as_numpy(raw_full_ds)

Let's print the first example.

In [ ]:
i = 0
print(f"Example Label: {full_labels[i]}")
print(f"Example Text: {raw_full_texts[i]}")

The data are stored as two numpy arrays:

1. `raw_full_texts` for the movie reviews in text format,
2. `full_labels` for the labels.


<div class="alert alert-info">
Bringing Your Own Data (BYOD)?

You can easily replace the above with your own tabular dataset, and continue with the rest of the tutorial.

</div>


Define a function to preprocess the text data by:

1. Converting it to lower case
2. Removing the HTML break tags: `<br />`
3. Removing any punctuation marks


In [ ]:
import tensorflow as tf
import re
import string


def preprocess_text(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(stripped_html, f"[{re.escape(string.punctuation)}]", "")

We use a `TextVectorization` layer to preprocess, tokenize, and vectorize our text data, thus making it suitable as input for a neural network.


In [ ]:
from tensorflow.keras import layers

tf.keras.utils.set_random_seed(SEED)

max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=preprocess_text,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

Adapting `vectorize_layer` to the text data creates a mapping of each token (i.e. word) to an integer index. Subsequently, we can vectorize our text data by using this mapping. Finally, we'll also convert our text data into a numpy array as required by `cleanlab`.


In [ ]:
%%capture

vectorize_layer.adapt(raw_full_texts)
full_texts = vectorize_layer(raw_full_texts)
full_texts = full_texts.numpy()

## **3. Define a classification model and compute out-of-sample predicted probabilities**


Here, we build a simple neural network for classification with TensorFlow and Keras.


In [ ]:
from tensorflow.keras import losses, metrics


def get_net():
    net = tf.keras.Sequential(
        [
            tf.keras.Input(shape=(None,), dtype="int64"),
            layers.Embedding(max_features + 1, 16),
            layers.Dropout(0.2),
            layers.GlobalAveragePooling1D(),
            layers.Dropout(0.2),
            layers.Dense(1),
        ]
    )  # outputs probability that text belongs to class 1

    net.compile(
        optimizer="adam",
        loss=losses.BinaryCrossentropy(from_logits=True),
        metrics=metrics.BinaryAccuracy(),
    )
    return net

As some of `cleanlab`'s feature requires scikit-learn compatibility, we will need to adapt the above TensorFlow & Keras neural net accordingly. [SciKeras](https://www.adriangb.com/scikeras/stable/) is a convenient package that makes this really easy.


In [ ]:
from scikeras.wrappers import KerasClassifier

model = KerasClassifier(get_net(), epochs=10)

To identify label issues, cleanlab requires a probabilistic prediction from your model for every datapoint that should be considered. However these predictions will be _overfit_ (and thus unreliable) for datapoints the model was previously trained on. `cleanlab` is intended to only be used with **out-of-sample** predicted probabilities, i.e. on datapoints held-out from the model during the training.

K-fold cross-validation is a straightforward way to produce out-of-sample predicted probabilites for every datapoint in the dataset, by training K copies of our model on different data subsets and using each copy to predict on the subset of data it did not see during training. We can obtain cross-validated out-of-sample predicted probabilities from any classifier via a scikit-learn simple wrapper:


In [ ]:
from sklearn.model_selection import cross_val_predict

num_crossval_folds = 3  # chosen for efficiency here, values like 5 or 10 will generally work better
pred_probs = cross_val_predict(
    model, full_texts, full_labels, cv=num_crossval_folds, method="predict_proba"
)

An additional benefit of cross-validation is that it facilitates more reliable evaluation of our model than a single training/validation split.


In [ ]:
from sklearn.metrics import roc_auc_score

auc = roc_auc_score(full_labels, pred_probs[:, 1])
print(f"Cross-validated estimate of held-out AUC score: {auc}")

## **4. Use cleanlab to find potential label errors**


Based on the given labels and out-of-sample predicted probabilities, `cleanlab` can quickly help us identify label issues. Here we request that the indices of the identified label issues should be sorted by `cleanlab`'s self-confidence score, which measures the quality of each given label via the probability assigned it in our model's prediction.


In [ ]:
from cleanlab.filter import find_label_issues

ranked_label_issues = find_label_issues(
    labels=full_labels, pred_probs=pred_probs, return_indices_ranked_by="self_confidence"
)

In [ ]:
# Hidden code cell to check if the highlighted examples that are shown later were found in the indices output by find_label_issues.
# If it is not found, throws an exception and breaks the docs build.
if not all(x in ranked_label_issues for x in [44582, 10404, 30151]):
    raise Exception("Highlighted examples not found in the array of label issues indices.")

Let's review some of the most likely label errors:


In [ ]:
print(
    f"cleanlab found {len(ranked_label_issues)} potential label errors. Here are indices of the top 10 most likely errors: \n {ranked_label_issues[:10]}"
)

To help us inspect these datapoints, we define a method to print any example from the dataset. We then display some of the top-ranked label issues identified by `cleanlab`:


In [ ]:
import pandas as pd

pd.set_option("display.max_colwidth", None)


def print_as_df(index):
    return pd.DataFrame({"texts": raw_full_texts[index], "labels": full_labels[index]}, [index])

Here's a review labeled as positive (1), but it should be negative (0).
Some noteworthy snippets extracted from the review text:

> - "...incredibly **awful** score..."
>
> - "...**worst** Foley work ever done."
>
> - "...script is **incomprehensible**..."
>
> - "...editing is just **bizarre**."
>
> - "...**atrocious** pan and scan..."
>
> - "...**incoherent mess**..."
>
> - "...**amateur** directing there."


In [ ]:
print_as_df(44582)

Here's a review labeled as positive (1), but it should be negative (0).
Some noteworthy snippets extracted from the review text:

> - "...film seems **cheap**."
>
> - "...unbelievably **bad**..."
>
> - "...cinematography is **badly** lit..."
>
> - "...everything looking **grainy** and **ugly**."
>
> - "...sound is so **terrible**..."


In [ ]:
print_as_df(10404)

Here's a review labeled as positive (1), but it should be negative (0).
Some noteworthy snippets extracted from the review text:

> - "...hard to imagine a **boring** shark movie..."
>
> - "**Poor focus** in some scenes made the production seems **amateurish**."
>
> - "...**do nothing** to take advantage of..."
>
> - "...**far too few** scenes of any depth or variety."
>
> - "...just **look flat**...no contrast of depth..."
>
> - "...**introspective** and **dull**...constant **disappointment**."


In [ ]:
print_as_df(30151)

`cleanlab` has shortlisted the most likely label errors to speed up your data cleaning process. With this list, you can decide whether to fix these label issues or remove ambiguous examples from the dataset.


## **5. Train a more robust model from noisy labels**


Fixing the label issues manually may be time-consuming, but at least `cleanlab` can filter these noisy examples and train a model on the remaining clean data for you automatically.
To demonstrate this, we first reload the dataset, this time with separate train and test splits.


In [ ]:
raw_train_ds = tfds.load(name="imdb_reviews", split="train", batch_size=-1, as_supervised=True)
raw_test_ds = tfds.load(name="imdb_reviews", split="test", batch_size=-1, as_supervised=True)

raw_train_texts, train_labels = tfds.as_numpy(raw_train_ds)
raw_test_texts, test_labels = tfds.as_numpy(raw_test_ds)

We featurize the raw text using the same `vectorize_layer` as before, but first, reset its state and adapt it only on the train set (as is proper ML practice). We finally convert the vectorized text data in the train/test sets into numpy arrays.


In [ ]:
vectorize_layer.reset_state()
vectorize_layer.adapt(raw_train_texts)

train_texts = vectorize_layer(raw_train_texts)
test_texts = vectorize_layer(raw_test_texts)

train_texts = train_texts.numpy()
test_texts = test_texts.numpy()

Let's now train and evaluate our original neural network model.


In [ ]:
from sklearn.metrics import accuracy_score

model = KerasClassifier(get_net(), epochs=10)
model.fit(train_texts, train_labels)

preds = model.predict(test_texts)
acc = accuracy_score(test_labels, preds)
print(f"\n Test acuracy of original neural net: {acc}")

`cleanlab` provides a wrapper class that can easily be applied to any scikit-learn compatible model. Once wrapped, the resulting model can still be used in the exact same manner, but it will now train more robustly if the data have noisy labels.


In [ ]:
from cleanlab.classification import LearningWithNoisyLabels

model = KerasClassifier(get_net(), epochs=10)  # Note we first re-instantiate the model
lnl = LearningWithNoisyLabels(clf=model, seed=SEED)  # lnl has same methods/attributes as model

When we train the `cleanlab`-wrapped model, the following operations take place: The original model is trained in a cross-validated fashion to produce out-of-sample predicted probabilities. Then, these predicted probabilites are used to identify label issues, which are then removed from the dataset. Finally, the original model is trained once more on the remaining clean subset of the data.


In [ ]:
_ = lnl.fit(train_texts, train_labels)

We can get predictions from the resulting cleanlab model and evaluate them, just like we did for our original neural network.


In [ ]:
pred_labels = lnl.predict(test_texts)
acc = accuracy_score(test_labels, pred_labels)
print(f"Test acuracy of cleanlab's neural net: {acc}")

We can see that the test set accuracy slightly improved as a result of the data cleaning. Note that this will not always be the case, especially when we are evaluating on test data that are themselves noisy. The best practice is to run `cleanlab` to identify potential label issues and then manually review them, before blindly trusting any accuracy metrics. In particular, the most effort should be made to ensure high-quality test data, which is supposed to reflect the expected performance of our model during deployment.
